<a href="https://colab.research.google.com/github/cdsouza2701/cdsouza2701/blob/main/Own_Notes_Exploring_a_SaaS_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect("saas.db")

##🔑 Working with SQL In Python

❌ SQL CANNOT run by itself in Colab

✅ SQL must be inside a Python function call

`pd.read_sql(""" SQL GOES HERE """, conn)`

🧠 Mental Model

- 🧑‍💼 SQL is the employee

- 🐍 Python is the manager

- 🏢 Colab only listens to the manager

So SQL must always be spoken through Python.

✅ Quick Checklist When You See This Error

- ❓ Is SQL code wrapped in """?

- ❓ Is it inside pd.read_sql(..., conn)?

- ❓ Has the cell that defines conn ran?

If any answer is “no” → error makes sense.

# Inspecting the Database

## 4.1 List all tables in database

In [ ]:
pd.read_sql("""
SELECT name
FROM sqlite_master
WHERE type='table';
""", conn)


,name
0,users
1,plans
2,subscriptions
3,sqlite_sequence
4,payments
5,events


## 4.2 Preview a table

In [ ]:
pd.read_sql("""
SELECT *
FROM users
LIMIT 5;
""", conn)

,user_id,signup_date,country,acquisition_channel
0,1,2023-04-08,US,partner
1,2,2025-04-27,NL,paid_social
2,3,2024-12-17,US,organic
3,4,2024-04-25,US,referral
4,5,2024-04-19,IN,organic


##4.3 Business Questions that could be answered by our database



### 4.3.1 User Table

| Column                | Meaning (business)    |
| --------------------- | --------------------- |
| `user_id`             | Unique person         |
| `signup_date`         | When they joined      |
| `country`             | Where they’re located |
| `acquisition_channel` | How they found us     |


#### 4.3.1.1 Growth Questions- User Table

1️⃣ Growth Questions (MOST IMPORTANT)

**Business question:**

Are we growing or slowing down?

**What you can learn:**

- User growth over time

- Seasonality

- Sudden drops or spikes

In [ ]:
pd.read_sql("""
SELECT
  strftime('%Y-%m', signup_date) AS month,
  COUNT(*) AS new_users
FROM users
GROUP BY 1
ORDER BY 1;
 """, conn)


,month,new_users
0,2023-01,57
1,2023-02,53
2,2023-03,44
3,2023-04,65
4,2023-05,66
5,2023-06,68
6,2023-07,55
7,2023-08,36
8,2023-09,61
9,2023-10,47




---



```
strftime('%Y-%m', signup_date) AS month
```

**`signup_date:`** A column from the users table.
Looks like: ***2024-03-18***

**`strftime('%Y-%m', signup_date)`**

This:

1. Takes a date

2. Extracts year and month

3. Turns it into text like: 2024-03



---



##### **Date querying:** Pulling ***features*** from a ***standardized date format*** that SQL can read.
---

SQL standard date format is: **`YYYY-MM-DD`**

---

**🧩 How SQLite (and SQL) See Dates**

SQLite does not have a true DATE type.

It stores dates as:

- TEXT

- INTEGER (timestamp)

- REAL (Julian date)

In your database, signup_date is stored as TEXT like this:

**YYYY-MM-DD**


Example:

2024-01-31


This format is not random.

---

**✅ Why *`YYYY-MM-DD`* Is Special**

This format is called ISO 8601, and it is:

- International

- Unambiguous

- Sorts correctly as text

- Understood by SQLite date functions

---

**🧠 Why The Queries Work Right Now**

The dataset already uses:

`YYYY-MM-DD`


That’s why this works:

`strftime('%Y-%m', signup_date)`


SQLite recognizes *signup_date* as a date string and can extract year/month.

---
**🧪 What Happens If Dates Are in 31.01.24 Format**

Let’s simulate a bad case:

`SELECT strftime('%Y', '31.01.24');`


Result:

`NULL`


SQLite says:

“I don’t know what this is.”

---

**🔧 How Businesses Handle Messy Date Formats:**

In real companies:

- Data often comes in messy formats eg.

- First step is **standardization**

  - Example: converting 31.01.24 → 2024-01-31.
  
    (The 31.01.24 is just a string and SQLite will not treat this as a date unless you manually parse it.)

  - This is usually done:

    - In ETL pipelines

    - In Python

    - Or with SQL transformations

    SQLite will not treat this as a date unless you manually parse it.

Example (advanced, just for awareness):

```
substr(date_text, 7, 4) || '-' ||
substr(date_text, 4, 2) || '-' ||
substr(date_text, 1, 2)
```



#####**🧠 Key Mental Model (This Will Save You Later)**

SQL date functions only work reliably if dates are stored in ISO format (YYYY-MM-DD).

Everything else is cleanup work.

##### **Understanding the format:**

🧠 The Big Idea (One Sentence)

The **%** sign means:

> “What follows is a special code, not normal text.”

That’s it.

---

**Think of It Like a Template (Very Important)**

Look at this:

`'%Y-%m'`


This is not a calculation.
It’s a pattern.

It's saying:

> “Replace %Y with the year and %m with the month.”

---

**🧠 Why % Needs Specific Letters**

Remember:

% means “the next character is a special instruction”

But the computer still needs to know which instruction

So strftime has a fixed dictionary of allowed codes.

If you use a letter it doesn’t know, it can’t guess.

**🧩 Valid strftime Codes (SQLite – the ones you need)**

Here are the important ones you’ll actually use:

| Code | Meaning         | Example |
| ---- | --------------- | ------- |
| `%Y` | Year (4 digits) | 2024    |
| `%y` | Year (2 digits) | 24      |
| `%m` | Month (01–12)   | 01      |
| `%d` | Day of month    | 31      |
| `%W` | Week of year    | 05      |
| `%H` | Hour (00–23)    | 14      |
| `%M` | Minute          | 32      |
| `%S` | Second          | 10      |

---

**✅ Summary (Only 3 Rules)**

- 1️⃣ % introduces a placeholder
- 2️⃣ The letter after % decides what
- 3️⃣ The result is a string


**🧠 One More Subtle Thing (Important Later)**

Different databases support different sets of codes.

Example:

- SQLite supports %Y

- PostgreSQL uses different functions entirely

But the idea stays the same.

In [ ]:
# See raw dates
pd.read_sql("""
SELECT signup_date
FROM users""", conn)

,signup_date
0,2023-04-08
1,2025-04-27
2,2024-12-17
3,2024-04-25
4,2024-04-19
...,...
1995,2023-01-03
1996,2024-08-28
1997,2025-12-23
1998,2023-11-04


In [ ]:
# See year only
pd.read_sql("""
SELECT signup_date, strftime('%Y', signup_date) AS Year
FROM users
""", conn)

,signup_date,Year
0,2023-04-08,2023
1,2025-04-27,2025
2,2024-12-17,2024
3,2024-04-25,2024
4,2024-04-19,2024
...,...,...
1995,2023-01-03,2023
1996,2024-08-28,2024
1997,2025-12-23,2025
1998,2023-11-04,2023


🧠 One Super Important Note (For Later)

Different databases have different date functions:

| Database   | Function        |
| ---------- | --------------- |
| SQLite     | `strftime`      |
| PostgreSQL | `DATE_TRUNC()`  |
| BigQuery   | `FORMAT_DATE()` |

`strftime` is short for:

**STR**ing **F**ormat **TIME**
